In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
from bayesian_testing.experiments import BinaryDataTest, DeltaLognormalDataTest

In [2]:
df = pd.read_csv("data/session_data.csv")

In [3]:
# example session data - each row represent one session
len(df)
df.head()

94500

conversion        date   revenue   source variant
0           0  2021-08-07  0.000000  desktop       B
1           1  2021-08-05  7.241015  desktop       C
2           0  2021-08-06  0.000000  desktop       A
3           0  2021-08-05  0.000000  desktop       C
4           0  2021-08-03  0.000000  desktop       A

In [4]:
# summary statistics per variant

summary = df.groupby('variant')[['variant', 'conversion', 'revenue']]\
            .agg({'variant': 'count', 'conversion': 'sum','revenue': 'sum'})\
            .rename(columns = {'variant': 'sessions', 'conversion': 'conversions'})

summary['conversion_rate'] = summary['conversions'] / summary['sessions']
summary['revenue_per_session'] = summary['revenue'] / summary['sessions']
summary['revenue_per_converted_sessions'] = summary['revenue'] / summary['conversions']

summary

sessions  conversions       revenue  conversion_rate  \
variant                                                         
A           31500         1580  30830.025613         0.050159   
B           32000         1700  35203.216888         0.053125   
C           31000         1550  37259.563364         0.050000   

         revenue_per_session  revenue_per_converted_sessions  
variant                                                       
A                   0.978731                       19.512674  
B                   1.100101                       20.707775  
C                   1.201921                       24.038428

# Conversion Test

In [5]:
conv_test = BinaryDataTest()

In [6]:
variant_A = df['conversion'][(df.variant == 'A')].values
variant_B = df['conversion'][(df.variant == 'B')].values
variant_C = df['conversion'][(df.variant == 'C')].values

variant_A
max(variant_A)

array([0, 0, 0, ..., 0, 0, 0])

1

In [7]:
conv_test.add_variant_data("A", variant_A)

In [8]:
conv_test.add_variant_data("B", variant_B)

In [9]:
conv_test.add_variant_data_agg("C", len(variant_C), sum(variant_C))

In [10]:
conv_test.data
conv_test.variant_names
conv_test.totals
conv_test.positives

{'A': {'totals': 31500, 'positives': 1580, 'a_prior': 0.5, 'b_prior': 0.5},
 'B': {'totals': 32000, 'positives': 1700, 'a_prior': 0.5, 'b_prior': 0.5},
 'C': {'totals': 31000, 'positives': 1550, 'a_prior': 0.5, 'b_prior': 0.5}}

['A', 'B', 'C']

[31500, 32000, 31000]

[1580, 1700, 1550]

In [11]:
conv_test.probabs_of_being_best(sim_count = 20000, seed=52)

{'A': 0.04185, 'B': 0.92235, 'C': 0.0358}

In [12]:
conv_test.expected_loss(sim_count = 20000, seed=52)

{'A': 0.0030138, 'B': 6.06e-05, 'C': 0.0031649}

In [13]:
conv_test.evaluate()

[{'variant': 'A',
  'totals': 31500,
  'positives': 1580,
  'positive_rate': 0.05016,
  'prob_being_best': 0.0405,
  'expected_loss': 0.0030217},
 {'variant': 'B',
  'totals': 32000,
  'positives': 1700,
  'positive_rate': 0.05312,
  'prob_being_best': 0.9273,
  'expected_loss': 5.47e-05},
 {'variant': 'C',
  'totals': 31000,
  'positives': 1550,
  'positive_rate': 0.05,
  'prob_being_best': 0.0322,
  'expected_loss': 0.0031855}]

# Revenue Test

In [14]:
rev_test = DeltaLognormalDataTest()

In [15]:
variant_A = df['revenue'][(df.variant == 'A')].values
variant_B = df['revenue'][(df.variant == 'B')].values
variant_C = df['revenue'][(df.variant == 'C')].values

variant_A
max(variant_A)

array([0., 0., 0., ..., 0., 0., 0.])

318.85100060418284

In [16]:
rev_test.add_variant_data("A", variant_A)
rev_test.add_variant_data("B", variant_B)

In [17]:
# adding variant using aggregated data
rev_test.add_variant_data_agg(
    name="C",
    totals=len(variant_C),
    positives=sum(x > 0 for x in variant_C),
    sum_values=sum(variant_C),
    sum_logs=sum([np.log(x) for x in variant_C if x > 0]),
    sum_logs_2=sum([np.square(np.log(x)) for x in variant_C if x > 0]),
)

In [18]:
# rev_test.data
rev_test.variant_names
rev_test.totals
rev_test.positives
rev_test.sum_values
rev_test.sum_logs
rev_test.sum_logs_2

['A', 'B', 'C']

[31500, 32000, 31000]

[1580, 1700, 1550]

[30830.02561293968, 35203.216888281, 37259.56336401027]

[3831.806394737818, 4211.729867679853, 4055.9652348481686]

[11029.923165846512, 12259.518683969136, 12357.91186291399]

In [19]:
rev_test.probabs_of_being_best()

{'A': 0.0002, 'B': 0.0326, 'C': 0.9672}

In [20]:
rev_test.expected_loss()

{'A': 0.22073, 'B': 0.1214353, 'C': 0.0008693}

In [21]:
rev_test.evaluate()

[{'variant': 'A',
  'totals': 31500,
  'positives': 1580,
  'sum_values': 30830.02561,
  'avg_values': 0.97873,
  'avg_positive_values': 19.51267,
  'prob_being_best': 5e-05,
  'expected_loss': 0.2197399},
 {'variant': 'B',
  'totals': 32000,
  'positives': 1700,
  'sum_values': 35203.21689,
  'avg_values': 1.1001,
  'avg_positive_values': 20.70777,
  'prob_being_best': 0.03285,
  'expected_loss': 0.1202885},
 {'variant': 'C',
  'totals': 31000,
  'positives': 1550,
  'sum_values': 37259.56336,
  'avg_values': 1.20192,
  'avg_positive_values': 24.03843,
  'prob_being_best': 0.9671,
  'expected_loss': 0.0007595}]

# Probabilities in time

In [22]:
# using time data we can generate probabilities of being best over time (cumulatively)

dates = np.sort(df.date.unique())
pbbs_conv_in_time = []
pbbs_rev_in_time = []
for i in dates:
    variant_A_conv = df[(df.variant == 'A') & (df.date <= i)]['conversion'].values
    variant_B_conv = df[(df.variant == 'B') & (df.date <= i)]['conversion'].values
    variant_C_conv = df[(df.variant == 'C') & (df.date <= i)]['conversion'].values
    
    variant_A_rev = df[(df.variant == 'A') & (df.date <= i)]['revenue'].values
    variant_B_rev = df[(df.variant == 'B') & (df.date <= i)]['revenue'].values
    variant_C_rev = df[(df.variant == 'C') & (df.date <= i)]['revenue'].values

    conv_test = BinaryDataTest()
    conv_test.add_variant_data("A", variant_A_conv)
    conv_test.add_variant_data("B", variant_B_conv)
    conv_test.add_variant_data("C", variant_C_conv)
    
    rev_test = DeltaLognormalDataTest()
    rev_test.add_variant_data("A", variant_A_rev)
    rev_test.add_variant_data("B", variant_B_rev)
    rev_test.add_variant_data("C", variant_C_rev)
    
    pbbs_conv_in_time.append({i: conv_test.probabs_of_being_best()})
    pbbs_rev_in_time.append({i: rev_test.probabs_of_being_best()})

In [23]:
pbbs_conv_in_time

[{'2021-08-01': {'A': 0.69095, 'B': 0.187, 'C': 0.12205}},
 {'2021-08-02': {'A': 0.37235, 'B': 0.21785, 'C': 0.4098}},
 {'2021-08-03': {'A': 0.1978, 'B': 0.6438, 'C': 0.1584}},
 {'2021-08-04': {'A': 0.15855, 'B': 0.743, 'C': 0.09845}},
 {'2021-08-05': {'A': 0.0326, 'B': 0.87125, 'C': 0.09615}},
 {'2021-08-06': {'A': 0.01505, 'B': 0.9712, 'C': 0.01375}},
 {'2021-08-07': {'A': 0.0409, 'B': 0.92325, 'C': 0.03585}}]

In [24]:
pbbs_rev_in_time

[{'2021-08-01': {'A': 0.0236, 'B': 0.26115, 'C': 0.71525}},
 {'2021-08-02': {'A': 0.05335, 'B': 0.21395, 'C': 0.7327}},
 {'2021-08-03': {'A': 0.0656, 'B': 0.20755, 'C': 0.72685}},
 {'2021-08-04': {'A': 0.0213, 'B': 0.1759, 'C': 0.8028}},
 {'2021-08-05': {'A': 0.00055, 'B': 0.0467, 'C': 0.95275}},
 {'2021-08-06': {'A': 0.0001, 'B': 0.04755, 'C': 0.95235}},
 {'2021-08-07': {'A': 0.0003, 'B': 0.03425, 'C': 0.96545}}]